# 概要
BigQueryの構文と対応したPandasの構文を記載する

## BigQueryテストデータ(単一テーブル)
単一テーブルのテストデータを以下の通り用意する。
```
#standardSQL
WITH
  A AS(
  SELECT
    *
  FROM
    UNNEST(ARRAY<STRUCT<dim1 STRING, dim2 STRING, dim3 STRING, mes1 INT64, mes2 FLOAT64>>[ 
      ('AAA', '001', 'val1', 100, 3.4),
      ('AAA', '002', 'val2', 200, 5.7),
      ('BBB', '003', 'val3', -100, NULL),
      ('CCC', '001', 'val4', NULL, -0.4),
      (NULL, '002',  'val5', 300, 5.4),
      ('DDD', NULL,  'val6', 400, 7.5),
      (NULL, NULL,   'val7', 500, 8.4)
      ]))
SELECT
  *
FROM
  A
```
![src_002.PNG](img/pandas_bigquery_cheetsheet/src_002.PNG)
以降はWITH句の記載を省略する。

In [30]:
# Pandasテストデータ(単一テーブル)
import numpy as np
import pandas as pd
# np.nan(NULL)があるときは、int型に指定できない
df_A = pd.DataFrame({'dim1' : pd.Series(['AAA', 'AAA', 'BBB', 'CCC', np.nan, 'DDD', np.nan], dtype='str'),
                     'dim2' : pd.Series(['001', '002', '003', '002', '003', np.nan, np.nan], dtype='str'),
                     'dim3' : pd.Series(['val1', 'val2', 'val3', 'val4', 'val5', 'val6', 'val7'], dtype='str'),
                     'mes1' : pd.Series([100, 200, -100, np.nan, 300, 400, 500], dtype='float64'), 
                     'mes2' : [3.4, 5.7, np.nan, -0.4, 5.4, 7.5, 8.4]})
df_A

,dim1,dim2,dim3,mes1,mes2
0,AAA,001,val1,100.0,3.4
1,AAA,002,val2,200.0,5.7
2,BBB,003,val3,-100.0,NaN
3,CCC,002,val4,NaN,-0.4
4,NaN,003,val5,300.0,5.4
5,DDD,NaN,val6,400.0,7.5
6,NaN,NaN,val7,500.0,8.4


In [33]:
# スキーマ情報など
df_A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dim1    5 non-null      object 
 1   dim2    5 non-null      object 
 2   dim3    7 non-null      object 
 3   mes1    6 non-null      float64
 4   mes2    6 non-null      float64
dtypes: float64(2), object(3)
memory usage: 408.0+ bytes


### Group By
BigQuery
```
SELECT
  dim1,
  sum(mes1) sum_mes1,
  COUNT(DISTINCT mes2) count_distinct_mes2,
  AVG(mes2) avg_mes2
FROM
  A
GROUP BY
  dim1
```

In [53]:
# Pandas
df_A.groupby(['dim1']).agg({'mes1' : 'sum', 'mes2' : [pd.Series.nunique, 'mean']}).reset_index()
# .reset_index()
# BBBはnp.nanなので、ユニーク数は0になる
# CCCのsum結果はBigQueryではnullだが、python的には0となる。lambda式で集計すればnanで出る
# dim1軸のnp.nanについては集計されない。集計対象としたい場合はfillnaで変換するのが無難

dim1   mes1    mes2      
          sum nunique  mean
0  AAA  300.0     2.0  4.55
1  BBB -100.0     0.0   NaN
2  CCC    0.0     1.0 -0.40
3  DDD  400.0     1.0  7.50